In [1]:
# The various libraries need to be imported to make the job much quicker
# These contain useful pre-written code
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import math
import random
import requests
import datetime
import csv
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys


# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.feature_extraction.text import CountVectorizer
# import nltk 
import string
import re
import os
import glob
import ast
import collections
# !pip install wordcloud
from wordcloud import WordCloud
# !pip install hazm
from hazm import *


%matplotlib inline

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows' , 100)
  

In [2]:
search_query = 'site:linkedin.com/in/ AND "python developer" AND "Germany"'


# username = input('Enter your Username ')
# password = input('Enter your password ')
# desired_job = input('Searched job query: ')
# desired_location = input('Location: ')

username = 'behdad.inkarnet@gmail.com'
password = 'behdad10973'
desired_job = 'data analyst'
desired_location = 'United Kingdom'

csv_file = str.strip(desired_job)
file_name = csv_file + "_" + str(desired_location) +'.csv'

In [ ]:
driver = webdriver.Firefox(executable_path= "geckodriver.exe")

profile = webdriver.FirefoxProfile()
profile.set_preference("browser.cache.disk.enable", False)
profile.set_preference("browser.cache.memory.enable", False)
profile.set_preference("browser.cache.offline.enable", False)
profile.set_preference("network.http.use-cache", False)
profile.update_preferences()

driver.get(url='https://www.linkedin.com/')
sleep(1)

# Find the sign in botthon
sign_in = driver.find_element(by='link text', value='Sign in')
sign_in.click()
sleep(3)

# We find the input section and enter our user information.
email_entry = driver.find_element(by='css selector', value='#username')
password_entry = driver.find_element(by='css selector', value='#password')

# For every letter in our email, we will set a random time between keystrokes
for letter in username:
    sleep(random.uniform(.1, .4))
    email_entry.send_keys(letter)

# We expect a random time between 0.1 and 0.4 sec for each letter of our password.
for letter in password:
    sleep(random.uniform(.1, .4))
    password_entry.send_keys(letter)

# We press enter
password_entry.send_keys(Keys.RETURN)
sleep(5)

search_bar = driver.find_element_by_id('jobs-tab-icon')
search_bar.click()
sleep(.5)



job_search_bar = driver.find_element_by_class_name("jobs-search-box__text-input")
job_search_bar.click()
sleep(1.5)

for letter in desired_job:
    sleep(random.uniform(.1, .4))
    job_search_bar.send_keys(letter)

job_search_bar.send_keys(Keys.RETURN)
sleep(3)

source = requests.get(driver.current_url).text
soup = BeautifulSoup(source, 'lxml')

i = 0
j = 0

writer = csv.writer(open(file_name, 'w', encoding='utf-8'))

writer.writerow(['company', 'job_title', 'job_description', 'employement_type ', 'industry', 'location', 'job_function',
                 'experience_Level' , 'time' , 'num_app' , 'num_view'])

while (j < 10):

    present_url = 'https://www.linkedin.com/jobs/search/?keywords=' + desired_job + '&location=' + desired_location + '&start=' + str(i)

    driver.get(present_url)
    scroll = driver.find_element_by_class_name('jobs-search-results')
    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)

    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)

    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)

    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)

    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)

    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)

    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)

    # ActionChains(driver).move_to_element(
    #     driver.find_element_by_class_name('search-results-pagination-section')).perform()

    jobs = driver.find_elements_by_class_name('job-card-search__title')

    for job in jobs:
        current_url = driver.current_url

        try:
            job.click()
        except:
            pass

        try:
            company_name = driver.find_element_by_class_name('jobs-details-top-card__company-url')
            company_name = company_name.text

            if company_name:
                company_name = company_name.strip()
        except:
            company_name = np.nan

        try:
            job_name = driver.find_element_by_class_name('jobs-details-top-card__job-title')
#             job_name = driver.find_element_by_class_name('jobs-details-top-card__job-title')
            job_name = job_name.text

            if job_name:
                job_name = job_name.strip()
        except:
            job_name = np.nan

        try:
            job_description = driver.find_element_by_class_name('jobs-description__container')
            if job_description =='':
                job_description = driver.find_element_by_class_name('jobs-box__html-content jobs-description-content__text')
#             job_description = driver.find_element_by_xpath('//*[@id="job-details"]')
            job_description = job_description.text

            if job_description:
                job_description = job_description.strip()
        except:
            job_description = np.nan

        try:
            employement_type = driver.find_element_by_class_name('js-formatted-employment-status-body')
            employement_type = employement_type.text

            if employement_type:
                employement_type = employement_type.strip()
        except:
            employement_type = np.nan

        try:
            industry = driver.find_element_by_class_name('js-formatted-industries-list')
            industry = industry.find_elements_by_class_name('jobs-box__list-item')
            industry = [ind.text for ind in industry]
            industry = ','.join(industry)

            if industry:
                industry = industry.strip()
        except:
            industry = np.nan

        try:
            location = driver.find_element_by_class_name('jobs-commute-module__company-location')
            if location =='':
                location = driver.find_element_by_class_name('jobs-details-top-card__bullet')
            location = location.text

            if location:
                location = location.strip()
        except:
            location = np.nan

        try:
            job_function = driver.find_element_by_class_name('js-formatted-job-functions-list')
            job_function = job_function.text

            if job_function:
                job_function = job_function.strip()
        except:
            job_function = np.nan

        try:
            experience_Level = driver.find_element_by_class_name('js-formatted-exp-body')
            experience_Level = experience_Level.text

            if experience_Level:
                experience_Level = experience_Level.strip()
        except:
            experience_Level = np.nan
            
        try:
            time = driver.find_elements_by_xpath("//*[contains(text(), 'Posted Date')]")
            time = time.text

            if time:
                time = time.strip()
        except:
            time = np.nan
            
            
        try:
            num_app = driver.find_element_by_class_name('jobs-details-job-summary__list-item display-flex align-items-center mb1 jobs-details-job-summary__list-item-bullet')
            if num_app =='':
                num_view = driver.driver.find_elements_by_xpath("//*[contains(text(), 'Number of views')]")
            
            num_app = num_app.text

            if num_app:
                num_app = num_app.strip()
        except:
            num_app = np.nan
            
        try:
            num_view = driver.driver.find_elements_by_xpath("//*[contains(text(), 'Number of views')]")
            
            num_view = num_view.text

            if num_view:
                num_view = num_view.strip()
        except:
            num_view = np.nan

        writer.writerow([company_name, job_name, job_description, employement_type, industry, location,
                         job_function, experience_Level , time , num_app , num_view, current_url])
        sleep(2)

    i = i + 25

    j = j + 1

print("The process is completed .../n The programis closing!")
driver.quit()

In [ ]:
df = pd.read_csv('data scientist.csv',index_col=False)
df

In [3]:
driver = webdriver.Firefox(executable_path= "geckodriver.exe")

profile = webdriver.FirefoxProfile()
profile.set_preference("browser.cache.disk.enable", False)
profile.set_preference("browser.cache.memory.enable", False)
profile.set_preference("browser.cache.offline.enable", False)
profile.set_preference("network.http.use-cache", False)
profile.update_preferences()

driver.get(url='https://www.linkedin.com/')
sleep(1)

# Find the sign in botthon
sign_in = driver.find_element(by='link text', value='Sign in')
sign_in.click()
sleep(3)

# We find the input section and enter our user information.
email_entry = driver.find_element(by='css selector', value='#username')
password_entry = driver.find_element(by='css selector', value='#password')

# For every letter in our email, we will set a random time between keystrokes
for letter in username:
    sleep(random.uniform(.1, .4))
    email_entry.send_keys(letter)

# We expect a random time between 0.1 and 0.4 sec for each letter of our password.
for letter in password:
    sleep(random.uniform(.1, .4))
    password_entry.send_keys(letter)

# We press enter
password_entry.send_keys(Keys.RETURN)
sleep(5)

search_bar = driver.find_element_by_id('jobs-tab-icon')
search_bar.click()
sleep(.5)



job_search_bar = driver.find_element_by_class_name("jobs-search-box__text-input")
job_search_bar.click()
sleep(3)

for letter in desired_job:
    sleep(random.uniform(.1, .4))
    job_search_bar.send_keys(letter)

job_search_bar.send_keys(Keys.RETURN)
sleep(3)

source = requests.get(driver.current_url).text
soup = BeautifulSoup(source, 'lxml')

i = 0
j = 0

writer = csv.writer(open(file_name, 'w', encoding='utf-8'))

writer.writerow(['company', 'job_title', 'job_description', 'employement_type ', 'industry', 'location', 'job_function',
                 'experience_Level' , 'num_app' ,'skill', 'job_link'])

while (j < 100):

    present_url = 'https://www.linkedin.com/jobs/search/?keywords=' + desired_job + '&location=' + desired_location + '&start=' + str(i)

    driver.get(present_url)
    scroll = driver.find_element_by_class_name('jobs-search-results')
    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)

    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)

    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)

    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)

    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)

    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)

    scroll.send_keys(Keys.PAGE_DOWN)

    sleep(1)


    jobs = driver.find_elements_by_class_name('job-card-search__title')

    for job in jobs:
        current_url = driver.current_url
        # Locate the anchor nodes first and load all the elements into some list
        lists = driver.find_elements_by_class_name('job-card-search__link-wrapper')
        # Empty list for storing links
        links = set()
        for lis in lists:
#             print(lis.get_attribute('href'))
            # Fetch and store the links
            links.add(lis.get_attribute('href'))
            
            # Loop through all the links and launch one by one
        for link in links:
            try:
                driver.get(link)
            except:
                pass
            
            
            # -------------Scrap here----------------
            
            print('---------------------------')
            print(link)
            ''' conpany _ title'''
            try:
                company_name = driver.find_element_by_class_name("jobs-top-card__company-url")
                if company_name == '' :
                    company_name = driver.find_element_by_class_name('jobs-top-card__company-info')
                company_name = company_name.text
                if company_name:
                    company_name = company_name.strip()
            except:
                company_name = np.nan
                
            print(company_name , '\n')
            sleep(0.5)
            
            ''' job _ title'''
            try:
                job_name = driver.find_element_by_class_name("jobs-top-card__job-title")
                job_name = job_name.text
                if job_name:
                    job_name = job_name.strip()
            except:
                job_name = np.nan
            
            print(job_name, '\n')
            sleep(0.5)
            
            ''' job _ description'''
            try:
                job_description = driver.find_element_by_class_name('jobs-description-content__text')
                if job_description == '':
                    job_description = driver.find_element_by_class_name('jobs-box__html-content')
                elif job_description == '' :
                    job_description = driver.find_element_by_id('job-details')
                job_description = job_description.text
                if job_description:
                    job_description = job_description.strip()
            except:
                    job_description = np.nan
                    
            print(job_description, '\n')
            sleep(0.5)
                    
            ''' employement _ type'''
            try:
                employement_type = driver.find_element_by_class_name("js-formatted-employment-status-body")
                employement_type = employement_type.text
                
                if employement_type:
                    employement_type = employement_type.strip()
            except:
                employement_type = np.nan
                
            print(employement_type, '\n')
            sleep(0.5)
            
            ''' industry'''
            try:
                industry = driver.find_element_by_class_name('js-formatted-industries-list')
                if industry == '' :
                    industry = driver.find_element_by_class_name("jobs-description-details__list")
                industry = industry.find_elements_by_class_name('jobs-box__list-item')
                industry = [ind.text for ind in industry]
                industry = ','.join(industry)
                
                if industry:
                    industry = industry.strip()
            except:
                industry = np.nan
                
            print(industry, '\n')
            sleep(0.5)
            
            ''' location'''
            try:
                location = driver.find_element_by_class_name("jobs-commute-module__company-location")
                if location == '':
                    location = driver.find_element_by_class_name("jobs-top-card__exact-location")
                location = location.text
                if location:
                    location = location.strip()
            except:
                location = np.nan
            
            print(location , '\n')
            sleep(0.5)
                
            ''' job_function'''
            try:
                job_function = driver.find_element_by_class_name("jobs-description-details__list-item")
                job_function = job_function.text
                if job_function:
                    job_function = job_function.strip()
            except:
                job_function = np.nan
                
            print(job_function , '\n')
            sleep(0.5)
                
            ''' experience_Level'''
            try:
                experience_Level = driver.find_element_by_class_name("js-formatted-exp-body")
                experience_Level = experience_Level.text
                if experience_Level:
                    experience_Level = experience_Level.strip()
            except:
                experience_Level = np.nan
                
            print(experience_Level , '\n')
            sleep(0.5)
                
            ''' num_app'''
            try:
                num_app = driver.find_element_by_class_name('ml1')
                if num_app == '':
                    num_app =driver.find_element_by_class_name('mt1')
                num_app = num_app.text
                
                if num_app:
                    num_app = num_app.strip()
            except:
                num_app = np.nan
                
            print(num_app , '\n')
            sleep(0.5)
            
            ''' skill'''
            try:
                skill = driver.find_element_by_class_name('jobs-ppc-criteria__list jobs-ppc-criteria__list--skills')
                skill = skill.text
                if skill:
                    skill = skill.strip()
            except:
                skill = np.nan
            
            print(skill , '\n')
            sleep(0.5)
            
            writer.writerow([company_name, job_name, job_description, employement_type, industry, location,
                         job_function, experience_Level , num_app ,skill, current_url])
            
            sleep(2)

    i = i + 25

    j = j + 1

print("The process is completed .../n The program is closing!")
driver.quit()



---------------------------
https://www.linkedin.com/jobs/view/1736692201/?eBP=JOB_SEARCH_ORGANIC&recommendedFlavor=HIDDEN_GEM&refId=1f7bd47c-a548-4ed9-a1b8-07dfce437a2d&trk=d_flagship3_search_srp_jobs
Mastercard 

Junior Analyst Data Science 

Who is Mastercard?

We are the global technology company behind the world’s fastest payments processing network. We are a vehicle for commerce, a connection to financial systems for the previously excluded, a technology innovation lab, and the home of Priceless®. We ensure every employee has the opportunity to be a part of something bigger and to change lives. We believe as our company grows, so should you. We believe in connecting everyone to endless, priceless possibilities.



Junior Analyst Data Science

Financial crime solutions team builds products and services powered by payments data. By combining data science technique with an intimate knowledge of payments data we develop solutions that will improve outcomes for people, businesses and 

 

 

 

nan 

nan 

nan 

 

nan 

 

nan 

---------------------------
https://www.linkedin.com/jobs/view/1730874363/?eBP=JOB_SEARCH_ORGANIC&refId=1f7bd47c-a548-4ed9-a1b8-07dfce437a2d&trk=d_flagship3_search_srp_jobs
 

 

 

nan 

nan 

nan 

 

 

nan 

nan 

---------------------------
https://www.linkedin.com/jobs/view/1702059225/?eBP=JOB_SEARCH_ORGANIC&refId=1f7bd47c-a548-4ed9-a1b8-07dfce437a2d&trk=d_flagship3_search_srp_jobs
 

 

 

nan 

nan 

nan 

nan 

nan 

 

nan 

---------------------------
https://www.linkedin.com/jobs/view/1730849929/?eBP=JOB_SEARCH_ORGANIC&refId=1f7bd47c-a548-4ed9-a1b8-07dfce437a2d&trk=d_flagship3_search_srp_jobs
 

 

 

nan 

nan 

nan 

nan 

nan 

 

nan 

---------------------------
https://www.linkedin.com/jobs/view/1736599187/?eBP=NotAvailableFromVoyagerAPI&recommendedFlavor=HIDDEN_GEM&refId=1f7bd47c-a548-4ed9-a1b8-07dfce437a2d&trk=d_flagship3_search_srp_jobs
nan 

 

 

nan 

nan 

nan 

 

 

nan 

nan 

---------------------------
https://w

InvalidSessionIdException: Message: Tried to run command without establishing a connection


## Data Explanatory analysis

In [6]:
df = pd.read_csv('data scientist_United Kingdom.csv',index_col=False)
df

,company,job_title,job_description,employement_type,industry,location,job_function,experience_Level,num_app,skill,job_link
0,Salt,Data Scientist,Data Scientist\r\nBooking platform (Saas)\r\n£...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.linkedin.com/jobs/search/?keywords...
1,"KBR, Inc.",Data Scientist,Technical Staffing Resources (TSR) are the in-...,NaN,NaN,NaN,NaN,NaN,131 applicants,NaN,https://www.linkedin.com/jobs/search/?keywords...
2,THG,Junior Data Scientist,Technical delivery: Must be able to deliver on...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.linkedin.com/jobs/search/?keywords...
3,Kleboe Jardine Ltd,Data Scientist,My client is a unique healthcare analytics and...,NaN,NaN,NaN,NaN,NaN,129 applicants,NaN,https://www.linkedin.com/jobs/search/?keywords...
4,Forsyth Barnes,Data Scientist,I am currently working exclusively with one of...,Full-time,"Marketing & Advertising,Staffing & Recruiting,...",NaN,Marketing & Advertising,Entry level,NaN,NaN,https://www.linkedin.com/jobs/search/?keywords...
...,...,...,...,...,...,...,...,...,...,...,...
744,E.ON UK,Data Analyst,"Location - Newstead Court, Nottingham NG15 0DR...",Full-time,"Information Technology & Services,Oil & Energy",NaN,Information Technology & Services,Mid-Senior level,35 applicants,NaN,https://www.linkedin.com/jobs/search/?keywords...
745,Sage,Data Engineer / Data Scientist,Advert Sage is a global company with a local h...,Full-time,"Information Technology & Services,Computer Sof...",NaN,Information Technology & Services,Mid-Senior level,NaN,NaN,https://www.linkedin.com/jobs/search/?keywords...
746,MBR Partners Limited,Data Scientist - Contract,The Role\r\n\r\nData Scientist (Contract/Full ...,NaN,"Computer Software,Telecommunications,Informati...",NaN,Computer Software,Mid-Senior level,33 applicants,NaN,https://www.linkedin.com/jobs/search/?keywords...
747,Nichols Digital Limited,Data Analyst,"Data Analyst, Quality Control, Data Analytics,...",Full-time,"Information Technology & Services,Computer Sof...",NaN,Information Technology & Services,Entry level,NaN,NaN,https://www.linkedin.com/jobs/search/?keywords...


In [7]:
jobs=df[~df['job_description'].isnull()]
jobs

,company,job_title,job_description,employement_type,industry,location,job_function,experience_Level,num_app,skill,job_link
0,Salt,Data Scientist,Data Scientist\r\nBooking platform (Saas)\r\n£...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.linkedin.com/jobs/search/?keywords...
1,"KBR, Inc.",Data Scientist,Technical Staffing Resources (TSR) are the in-...,NaN,NaN,NaN,NaN,NaN,131 applicants,NaN,https://www.linkedin.com/jobs/search/?keywords...
2,THG,Junior Data Scientist,Technical delivery: Must be able to deliver on...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.linkedin.com/jobs/search/?keywords...
3,Kleboe Jardine Ltd,Data Scientist,My client is a unique healthcare analytics and...,NaN,NaN,NaN,NaN,NaN,129 applicants,NaN,https://www.linkedin.com/jobs/search/?keywords...
4,Forsyth Barnes,Data Scientist,I am currently working exclusively with one of...,Full-time,"Marketing & Advertising,Staffing & Recruiting,...",NaN,Marketing & Advertising,Entry level,NaN,NaN,https://www.linkedin.com/jobs/search/?keywords...
...,...,...,...,...,...,...,...,...,...,...,...
744,E.ON UK,Data Analyst,"Location - Newstead Court, Nottingham NG15 0DR...",Full-time,"Information Technology & Services,Oil & Energy",NaN,Information Technology & Services,Mid-Senior level,35 applicants,NaN,https://www.linkedin.com/jobs/search/?keywords...
745,Sage,Data Engineer / Data Scientist,Advert Sage is a global company with a local h...,Full-time,"Information Technology & Services,Computer Sof...",NaN,Information Technology & Services,Mid-Senior level,NaN,NaN,https://www.linkedin.com/jobs/search/?keywords...
746,MBR Partners Limited,Data Scientist - Contract,The Role\r\n\r\nData Scientist (Contract/Full ...,NaN,"Computer Software,Telecommunications,Informati...",NaN,Computer Software,Mid-Senior level,33 applicants,NaN,https://www.linkedin.com/jobs/search/?keywords...
747,Nichols Digital Limited,Data Analyst,"Data Analyst, Quality Control, Data Analytics,...",Full-time,"Information Technology & Services,Computer Sof...",NaN,Information Technology & Services,Entry level,NaN,NaN,https://www.linkedin.com/jobs/search/?keywords...


In [9]:
jobs.columns

Index(['company', 'job_title', 'job_description', 'employement_type ',
       'industry', 'location', 'job_function', 'experience_Level', 'num_app',
       'skill', 'job_link'],
      dtype='object')

In [10]:
jobs.dtypes

company               object
job_title             object
job_description       object
employement_type      object
industry              object
location              object
job_function          object
experience_Level      object
num_app               object
skill                float64
job_link              object
dtype: object

In [11]:
del jobs['skill']

In [12]:
jobs

,company,job_title,job_description,employement_type,industry,location,job_function,experience_Level,num_app,job_link
0,Salt,Data Scientist,Data Scientist\r\nBooking platform (Saas)\r\n£...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.linkedin.com/jobs/search/?keywords...
1,"KBR, Inc.",Data Scientist,Technical Staffing Resources (TSR) are the in-...,NaN,NaN,NaN,NaN,NaN,131 applicants,https://www.linkedin.com/jobs/search/?keywords...
2,THG,Junior Data Scientist,Technical delivery: Must be able to deliver on...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.linkedin.com/jobs/search/?keywords...
3,Kleboe Jardine Ltd,Data Scientist,My client is a unique healthcare analytics and...,NaN,NaN,NaN,NaN,NaN,129 applicants,https://www.linkedin.com/jobs/search/?keywords...
4,Forsyth Barnes,Data Scientist,I am currently working exclusively with one of...,Full-time,"Marketing & Advertising,Staffing & Recruiting,...",NaN,Marketing & Advertising,Entry level,NaN,https://www.linkedin.com/jobs/search/?keywords...
...,...,...,...,...,...,...,...,...,...,...
744,E.ON UK,Data Analyst,"Location - Newstead Court, Nottingham NG15 0DR...",Full-time,"Information Technology & Services,Oil & Energy",NaN,Information Technology & Services,Mid-Senior level,35 applicants,https://www.linkedin.com/jobs/search/?keywords...
745,Sage,Data Engineer / Data Scientist,Advert Sage is a global company with a local h...,Full-time,"Information Technology & Services,Computer Sof...",NaN,Information Technology & Services,Mid-Senior level,NaN,https://www.linkedin.com/jobs/search/?keywords...
746,MBR Partners Limited,Data Scientist - Contract,The Role\r\n\r\nData Scientist (Contract/Full ...,NaN,"Computer Software,Telecommunications,Informati...",NaN,Computer Software,Mid-Senior level,33 applicants,https://www.linkedin.com/jobs/search/?keywords...
747,Nichols Digital Limited,Data Analyst,"Data Analyst, Quality Control, Data Analytics,...",Full-time,"Information Technology & Services,Computer Sof...",NaN,Information Technology & Services,Entry level,NaN,https://www.linkedin.com/jobs/search/?keywords...


In [14]:
missing_data = jobs.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print("\n")

company
False    713
True       2
Name: company, dtype: int64


job_title
False    715
Name: job_title, dtype: int64


job_description
False    715
Name: job_description, dtype: int64


employement_type 
False    373
True     342
Name: employement_type , dtype: int64


industry
False    564
True     151
Name: industry, dtype: int64


location
True     676
False     39
Name: location, dtype: int64


job_function
False    519
True     196
Name: job_function, dtype: int64


experience_Level
False    477
True     238
Name: experience_Level, dtype: int64


num_app
True     450
False    265
Name: num_app, dtype: int64


job_link
False    715
Name: job_link, dtype: int64




In [17]:
jobs['company'].value_counts()

Harnham                                         3
Burberry                                        3
Signifyd                                        3
Mastercard                                      3
Covance                                         2
                                               ..
Bohemian Rhapsody Silver                        1
Cognitive Group | Microsoft Talent Solutions    1
WhatsApp Inc.                                   1
Vodafone                                        1
NOW TV                                          1
Name: company, Length: 612, dtype: int64

In [ ]:
# discription_All = " ".join(str(discription) for discription in jobs.job_description)

In [27]:
print(len(jobs['job_title'].unique()) , ' Unique Title')
jobs['job_title'].value_counts()

188  Unique Title


Data Scientist                                                          235
Data Analyst                                                            174
Senior Data Scientist                                                    40
Lead Data Scientist                                                      17
Junior Data Scientist                                                    16
                                                                       ... 
Performance Data Analyst                                                  1
Bioinformatician/Data Scientist                                           1
Data Analyst - Undergraduate Placement Competitive Salary + Benefits      1
Data Science Intern                                                       1
Forensic Data Analyst                                                     1
Name: job_title, Length: 188, dtype: int64

In [1]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.tight_layout()

jobs['job_title'].value_counts().nlargest(15).plot(kind='bar')


NameError: name 'jobs' is not defined

In [51]:
discription_df = jobs[['job_description']].copy()
discription_df

,job_description
0,"Data Scientist\r\nBooking platform (Saas)\r\n£55,000 - £65,000 basic plus bens\r\nLondon, (with ..."
1,Technical Staffing Resources (TSR) are the in-house agency and master vendor for KBR who are a l...
2,"Technical delivery: Must be able to deliver on a technical level. You will be writing code, prod..."
3,My client is a unique healthcare analytics and strategy consultancy organisation based in the Sc...
4,I am currently working exclusively with one of UK's fastest growing tech\r\ncompanies who are of...
...,...
744,"Location - Newstead Court, Nottingham NG15 0DR\r\n\r\nSalary - £49,048 - £65,980\r\n\r\nThe Tech..."
745,Advert Sage is a global company with a local heart. The market leader for cloud-based accounting...
746,The Role\r\n\r\nData Scientist (Contract/Full time)\r\n\r\nWho We Are Looking For People\r\nStro...
747,"Data Analyst, Quality Control, Data Analytics, Excel, either PHP/ Java/\r\nSQL/ MATLAB/ Yellowfi..."


In [52]:
discription_df.isna().sum()

job_description    0
dtype: int64

In [53]:
rand_indexs = np.random.randint(1,len(discription_df),50).tolist()
discription_df["job_description"][rand_indexs]

C:\Users\behdad\Anaconda3\lib\site-packages\pandas\core\series.py:1143: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


328    ABOUT WHITEHAT ANALYTICS\r\n\r\nWhitehat Analytics is a boutique data science consultancy specia...
645    Data Analyst\r\nLondon\r\n\r\nOpportunity for a Data Analyst to join and internationally renowne...
158    Data Analyst | Medtech Start-up | SQL, Python | £50,000-55,000\r\n  Our client, a successful, fa...
91     Facebook's mission is to give people the power to build community and bring the world closer tog...
585                                                                                                    NaN
240    The Role\r\n\r\nData Scientist (Permanent)\r\n\r\nWho We Are Looking For People\r\nStrong analyt...
319    Are you a logical problem solver? Do you enjoy solving complex puzzles but explaining results in...
79     Burberry is making a major investment in expanding our data and analytics capability. Over the l...
543    Key Group currently have a fantastic graduate opportunity for a Data & BI Analyst to join their ...
164    Data Scientist\r\n\r\nLondon\r

In [58]:
import string
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = re.sub(r'[^a-zA-Z0-9 \n\.]', ' ' , text)
    return text

discription_df['description_punct'] = discription_df["job_description"].apply(lambda x: remove_punct(x))
discription_df.head(10)

,job_description,description_punct
0,"Data Scientist\r\nBooking platform (Saas)\r\n£55,000 - £65,000 basic plus bens\r\nLondon, (with ...",Data Scientist Booking platform Saas basic plus bens London with some flexible working ...
1,Technical Staffing Resources (TSR) are the in-house agency and master vendor for KBR who are a l...,Technical Staffing Resources TSR are the inhouse agency and master vendor for KBR who are a lead...
2,"Technical delivery: Must be able to deliver on a technical level. You will be writing code, prod...",Technical delivery Must be able to deliver on a technical level You will be writing code produce...
3,My client is a unique healthcare analytics and strategy consultancy organisation based in the Sc...,My client is a unique healthcare analytics and strategy consultancy organisation based in the Sc...
4,I am currently working exclusively with one of UK's fastest growing tech\r\ncompanies who are of...,I am currently working exclusively with one of UKs fastest growing tech companies who are offic...
5,Data Scientist Opportunity\r\n\r\nWe are a global advisor enabling organizations to reimagine th...,Data Scientist Opportunity We are a global advisor enabling organizations to reimagine their ...
6,"Data Scientist\r\n\r\nLOCATION:\r\nLondon, LND, GB\r\n\r\n\r\nJOB CATEGORY: Marketing\r\n\r\nREQ...",Data Scientist LOCATION London LND GB JOB CATEGORY Marketing REQUISITION NUMBER ...
7,"Playdemic, a division of WB Games Inc., are looking for a talented data scientist with a passion...",Playdemic a division of WB Games Inc are looking for a talented data scientist with a passion fo...
8,Position Overview\r\n\r\nGroupM is the leading global media investment company serving as the pa...,Position Overview GroupM is the leading global media investment company serving as the parent...
9,Are we right for each other?\r\n Flamingo is a global cultural fluency agency within the Omnico...,Are we right for each other Flamingo is a global cultural fluency agency within the Omnicom G...


In [60]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

discription_df['discription_tokenized'] = discription_df['description_punct'].apply(lambda x: tokenization(x.lower()))
discription_df.head()

,job_description,description_punct,discription_tokenized
0,"Data Scientist\r\nBooking platform (Saas)\r\n£55,000 - £65,000 basic plus bens\r\nLondon, (with ...",Data Scientist Booking platform Saas basic plus bens London with some flexible working ...,"[data, scientist, booking, platform, saas, basic, plus, bens, london, with, some, flexible, work..."
1,Technical Staffing Resources (TSR) are the in-house agency and master vendor for KBR who are a l...,Technical Staffing Resources TSR are the inhouse agency and master vendor for KBR who are a lead...,"[technical, staffing, resources, tsr, are, the, inhouse, agency, and, master, vendor, for, kbr, ..."
2,"Technical delivery: Must be able to deliver on a technical level. You will be writing code, prod...",Technical delivery Must be able to deliver on a technical level You will be writing code produce...,"[technical, delivery, must, be, able, to, deliver, on, a, technical, level, you, will, be, writi..."
3,My client is a unique healthcare analytics and strategy consultancy organisation based in the Sc...,My client is a unique healthcare analytics and strategy consultancy organisation based in the Sc...,"[my, client, is, a, unique, healthcare, analytics, and, strategy, consultancy, organisation, bas..."
4,I am currently working exclusively with one of UK's fastest growing tech\r\ncompanies who are of...,I am currently working exclusively with one of UKs fastest growing tech companies who are offic...,"[i, am, currently, working, exclusively, with, one, of, uks, fastest, growing, tech, companies, ..."


In [62]:
import nltk
stopword = nltk.corpus.stopwords.words('english')

In [63]:
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
discription_df['discription_nonstop'] = discription_df['discription_tokenized'].apply(lambda x: remove_stopwords(x))
discription_df.head(10)

,job_description,description_punct,discription_tokenized,discription_nonstop
0,"Data Scientist\r\nBooking platform (Saas)\r\n£55,000 - £65,000 basic plus bens\r\nLondon, (with ...",Data Scientist Booking platform Saas basic plus bens London with some flexible working ...,"[data, scientist, booking, platform, saas, basic, plus, bens, london, with, some, flexible, work...","[data, scientist, booking, platform, saas, basic, plus, bens, london, flexible, working, role, n..."
1,Technical Staffing Resources (TSR) are the in-house agency and master vendor for KBR who are a l...,Technical Staffing Resources TSR are the inhouse agency and master vendor for KBR who are a lead...,"[technical, staffing, resources, tsr, are, the, inhouse, agency, and, master, vendor, for, kbr, ...","[technical, staffing, resources, tsr, inhouse, agency, master, vendor, kbr, leading, global, eng..."
2,"Technical delivery: Must be able to deliver on a technical level. You will be writing code, prod...",Technical delivery Must be able to deliver on a technical level You will be writing code produce...,"[technical, delivery, must, be, able, to, deliver, on, a, technical, level, you, will, be, writi...","[technical, delivery, must, able, deliver, technical, level, writing, code, produce, reports, im..."
3,My client is a unique healthcare analytics and strategy consultancy organisation based in the Sc...,My client is a unique healthcare analytics and strategy consultancy organisation based in the Sc...,"[my, client, is, a, unique, healthcare, analytics, and, strategy, consultancy, organisation, bas...","[client, unique, healthcare, analytics, strategy, consultancy, organisation, based, scottish, bo..."
4,I am currently working exclusively with one of UK's fastest growing tech\r\ncompanies who are of...,I am currently working exclusively with one of UKs fastest growing tech companies who are offic...,"[i, am, currently, working, exclusively, with, one, of, uks, fastest, growing, tech, companies, ...","[currently, working, exclusively, one, uks, fastest, growing, tech, companies, official, partner..."
5,Data Scientist Opportunity\r\n\r\nWe are a global advisor enabling organizations to reimagine th...,Data Scientist Opportunity We are a global advisor enabling organizations to reimagine their ...,"[data, scientist, opportunity, we, are, a, global, advisor, enabling, organizations, to, reimagi...","[data, scientist, opportunity, global, advisor, enabling, organizations, reimagine, future, crea..."
6,"Data Scientist\r\n\r\nLOCATION:\r\nLondon, LND, GB\r\n\r\n\r\nJOB CATEGORY: Marketing\r\n\r\nREQ...",Data Scientist LOCATION London LND GB JOB CATEGORY Marketing REQUISITION NUMBER ...,"[data, scientist, location, london, lnd, gb, job, category, marketing, requisition, number, kins...","[data, scientist, location, london, lnd, gb, job, category, marketing, requisition, number, kins..."
7,"Playdemic, a division of WB Games Inc., are looking for a talented data scientist with a passion...",Playdemic a division of WB Games Inc are looking for a talented data scientist with a passion fo...,"[playdemic, a, division, of, wb, games, inc, are, looking, for, a, talented, data, scientist, wi...","[playdemic, division, wb, games, inc, looking, talented, data, scientist, passion, mobile, games..."
8,Position Overview\r\n\r\nGroupM is the leading global media investment company serving as the pa...,Position Overview GroupM is the leading global media investment company serving as the parent...,"[position, overview, groupm, is, the, leading, global, media, investment, company, serving, as, ...","[position, overview, groupm, leading, global, media, investment, company, serving, parent, wpp, ..."
9,Are we right for each other?\r\n Flamingo is a global cultural fluency agency within the Omnico...,Are we right for each other Flamingo is a global cultural fluency agency within the Omnicom G...,"[are, we, right, for, each, other, flamingo, is, a, global, cultural, fluency, agency, within,

In [65]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

discription_df['discription_stemmed'] = discription_df['discription_nonstop'].apply(lambda x: stemming(x))
discription_df.head()

,job_description,description_punct,discription_tokenized,discription_nonstop,discription_stemmed
0,"Data Scientist\r\nBooking platform (Saas)\r\n£55,000 - £65,000 basic plus bens\r\nLondon, (with ...",Data Scientist Booking platform Saas basic plus bens London with some flexible working ...,"[data, scientist, booking, platform, saas, basic, plus, bens, london, with, some, flexible, work...","[data, scientist, booking, platform, saas, basic, plus, bens, london, flexible, working, role, n...","[data, scientist, book, platform, saa, basic, plu, ben, london, flexibl, work, role, newli, crea..."
1,Technical Staffing Resources (TSR) are the in-house agency and master vendor for KBR who are a l...,Technical Staffing Resources TSR are the inhouse agency and master vendor for KBR who are a lead...,"[technical, staffing, resources, tsr, are, the, inhouse, agency, and, master, vendor, for, kbr, ...","[technical, staffing, resources, tsr, inhouse, agency, master, vendor, kbr, leading, global, eng...","[technic, staf, resourc, tsr, inhous, agenc, master, vendor, kbr, lead, global, engin, construct..."
2,"Technical delivery: Must be able to deliver on a technical level. You will be writing code, prod...",Technical delivery Must be able to deliver on a technical level You will be writing code produce...,"[technical, delivery, must, be, able, to, deliver, on, a, technical, level, you, will, be, writi...","[technical, delivery, must, able, deliver, technical, level, writing, code, produce, reports, im...","[technic, deliveri, must, abl, deliv, technic, level, write, code, produc, report, implement, hi..."
3,My client is a unique healthcare analytics and strategy consultancy organisation based in the Sc...,My client is a unique healthcare analytics and strategy consultancy organisation based in the Sc...,"[my, client, is, a, unique, healthcare, analytics, and, strategy, consultancy, organisation, bas...","[client, unique, healthcare, analytics, strategy, consultancy, organisation, based, scottish, bo...","[client, uniqu, healthcar, analyt, strategi, consult, organis, base, scottish, border, commercia..."
4,I am currently working exclusively with one of UK's fastest growing tech\r\ncompanies who are of...,I am currently working exclusively with one of UKs fastest growing tech companies who are offic...,"[i, am, currently, working, exclusively, with, one, of, uks, fastest, growing, tech, companies, ...","[currently, working, exclusively, one, uks, fastest, growing, tech, companies, official, partner...","[current, work, exclus, one, uk, fastest, grow, tech, compani, offici, partner, facebook, googl,..."


In [66]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

discription_df['discription_lemmatized'] = discription_df['discription_nonstop'].apply(lambda x: lemmatizer(x))
discription_df.head()

,job_description,description_punct,discription_tokenized,discription_nonstop,discription_stemmed,discription_lemmatized
0,"Data Scientist\r\nBooking platform (Saas)\r\n£55,000 - £65,000 basic plus bens\r\nLondon, (with ...",Data Scientist Booking platform Saas basic plus bens London with some flexible working ...,"[data, scientist, booking, platform, saas, basic, plus, bens, london, with, some, flexible, work...","[data, scientist, booking, platform, saas, basic, plus, bens, london, flexible, working, role, n...","[data, scientist, book, platform, saa, basic, plu, ben, london, flexibl, work, role, newli, crea...","[data, scientist, booking, platform, saas, basic, plus, ben, london, flexible, working, role, ne..."
1,Technical Staffing Resources (TSR) are the in-house agency and master vendor for KBR who are a l...,Technical Staffing Resources TSR are the inhouse agency and master vendor for KBR who are a lead...,"[technical, staffing, resources, tsr, are, the, inhouse, agency, and, master, vendor, for, kbr, ...","[technical, staffing, resources, tsr, inhouse, agency, master, vendor, kbr, leading, global, eng...","[technic, staf, resourc, tsr, inhous, agenc, master, vendor, kbr, lead, global, engin, construct...","[technical, staffing, resource, tsr, inhouse, agency, master, vendor, kbr, leading, global, engi..."
2,"Technical delivery: Must be able to deliver on a technical level. You will be writing code, prod...",Technical delivery Must be able to deliver on a technical level You will be writing code produce...,"[technical, delivery, must, be, able, to, deliver, on, a, technical, level, you, will, be, writi...","[technical, delivery, must, able, deliver, technical, level, writing, code, produce, reports, im...","[technic, deliveri, must, abl, deliv, technic, level, write, code, produc, report, implement, hi...","[technical, delivery, must, able, deliver, technical, level, writing, code, produce, report, imp..."
3,My client is a unique healthcare analytics and strategy consultancy organisation based in the Sc...,My client is a unique healthcare analytics and strategy consultancy organisation based in the Sc...,"[my, client, is, a, unique, healthcare, analytics, and, strategy, consultancy, organisation, bas...","[client, unique, healthcare, analytics, strategy, consultancy, organisation, based, scottish, bo...","[client, uniqu, healthcar, analyt, strategi, consult, organis, base, scottish, border, commercia...","[client, unique, healthcare, analytics, strategy, consultancy, organisation, based, scottish, bo..."
4,I am currently working exclusively with one of UK's fastest growing tech\r\ncompanies who are of...,I am currently working exclusively with one of UKs fastest growing tech companies who are offic...,"[i, am, currently, working, exclusively, with, one, of, uks, fastest, growing, tech, companies, ...","[currently, working, exclusively, one, uks, fastest, growing, tech, companies, official, partner...","[current, work, exclus, one, uk, fastest, grow, tech, compani, offici, partner, facebook, googl,...","[currently, working, exclusively, one, uk, fastest, growing, tech, company, official, partner, f..."


In [3]:
from nltk.tokenize import word_tokenize

# Uncomment this line if you haven't downloaded punkt before
# or just run it as it is and uncomment it if you got an error.
# nltk.download('punkt')

def most_used_words(text):
#     tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(text)
    print("There is %d different words" % len(set(text)))
    return sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)

In [6]:
# words = set(nltk.corpus.words.words())

# test_text_data = " ".join(w for w in nltk.wordpunct_tokenize(discription_df.job_description.str.cat()) \
#          if w.lower() in words or not w.isalpha())
test_text_data = " ".join(w for w in discription_df['discription_tokenized'].str.cat()) 

NameError: name 'discription_df' is not defined

In [4]:
# most_used_words(test_text_data)[:10]
most_used_words(discription_df['discription_tokenized'])[:10]


NameError: name 'discription_df' is not defined

In [81]:
from nltk.corpus import stopwords

# nltk.download("stopwords")

mw = most_used_words(discription_df.job_description.str.cat())
most_words = []
for w in mw:
    if len(most_words) == 1000:
        break
    if w in stopwords.words("english"):
        continue
    else:
        most_words.append(w)

There is 11333 different words


In [82]:
sorted(most_words)

['!',
 '#',
 '$',
 '%',
 '&',
 "'",
 "''",
 "'ll",
 "'re",
 "'s",
 "'ve",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '...',
 '/',
 '1',
 '10',
 '100',
 '2',
 '2018',
 '2019',
 '2020',
 '25',
 '3',
 '5',
 ':',
 ';',
 '?',
 '@',
 'A',
 'AI',
 'Ability',
 'About',
 'All',
 'An',
 'Analysis',
 'Analyst',
 'Analytics',
 'And',
 'Apply',
 'Are',
 'As',
 'At',
 'Attest',
 'Azure',
 'BI',
 'Be',
 'Benefits',
 'Best',
 'Big',
 'Business',
 'But',
 'By',
 'CV',
 'Cambridge',
 'Central',
 'Company',
 'Competitive',
 'Computer',
 'Customer',
 'D',
 'Data',
 'Date',
 'Deloitte',
 'Description',
 'Development',
 'Digital',
 'Do',
 'Employment',
 'Engineering',
 'Engineers',
 'England',
 'Europe',
 'Excel',
 'Excellent',
 'Experience',
 'Finance',
 'Financial',
 'Fitch',
 'For',
 'From',
 'GSK',
 'Global',
 'Graduate',
 'Group',
 'Head',
 'Health',
 'I',
 'IT',
 'If',
 'In',
 'Indicators',
 'Information',
 'Insight',
 'Intelligence',
 'It',
 'Job',
 'Join',
 'Junior',
 'Key',
 'Knowledge',
 'Lead',
 

In [83]:
# I'm defining this function to use it in the 
# Data Preparation Phase
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

# nltk.download('wordnet')
def stem_tokenize(text):
    stemmer = SnowballStemmer("english")
    stemmer = WordNetLemmatizer()
    return [stemmer.lemmatize(token) for token in word_tokenize(text)]

def lemmatize_tokenize(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(text)]

In [84]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
# Start with one review:
text = " ".join(discription for discription in discription_df.description_punct)
# lower max_font_size, change the maximum number of word and lighten the background:
stopwords = set(STOPWORDS)
# stopwords.add(search_modifier)
wordcloud = WordCloud(max_font_size=50, max_words=100,stopwords= stopwords ,  background_color="white").generate(text)


plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()
plt.savefig('1.png')

## Frequency Analysis

In [85]:
#download nltk attribute if needed
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('reuters')

[nltk_data] Downloading package reuters to C:\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True